<a href="https://colab.research.google.com/github/marcory-hub/yolo11n-on-grove-vision-ai-v2/blob/main/YOLO11_pt_to_vela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO11n full integer quantisation and vela conversion for Grove Vision AI V2


Check if you have to replace the default options with your custom model name and image size:
- `imgsz`=224 (or 192, default for Grove vision AI V2, or other sizes. Train the yolo model with the imgsz that you are going to use on the Grove Vision AI V2.)
- `model_name`=best.pt (or replace best with a custom name)
- `nc` = 4 (or your number of classes, fe coco dataset has 80 classes)
- `class_names`='amel', 'vcra', 'vespsp', 'vvel' (or your default classnames, in this format)
- `name`=best (or your custom name)

1. Make sure images and labels from your dataset have this folder structure with these exact names. And add `data.yaml` to main folder.

```
🗂️ dataset
  🗂️ train
    🗂️ images
    🗂️ labels
  🗂️ valid
    🗂️ images
    🗂️ labels
  data.yaml
```

2. Zip the dataset folder to a file names `dataset.zip` On mac use `zip -r dataset.zip . -x "*.DS_Store" "__MACOSX/*" ".Trashes/*" ".Spotlight-V100/*" ".TemporaryItems/*"` to exclude hidden files, such as finderfiles, from the zipped file.

3. Make in /content/drive/MyDrive the folder `yolo`. Copy the `dataset.zip` file to this folder, it is needed to make a callibration image set and your yolo model, fe `best.pt` to this folder. For the model you can use a custom name and adjust it in the options below.

4. Copy the `dataset.zip` file to the folder /content/drive/MyDrive/yolo.



This notebook uses the module imp. It is no longer available in python 3.12 (depricated since python 3.4). As workaround the code runs in a python 3.10 env.

In [ ]:
model_name = "best2.pt"
print(f"Variable defined: {model_name}")

## Data preparation

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Copy zipped dataset to colab and unzip the dataset
!scp '/content/drive/MyDrive/yolo/dataset.zip' '/content/dataset.zip'
!unzip '/content/dataset.zip' -d '/content/'

In [ ]:
# Make a temp_dataset from the validation images for calibration purposes
import os
import random
import shutil
import yaml

nc = "4" #@param {type:"string"}
class_names = "'amel', 'vcra', 'vespsp', 'vvel'" #@param {type:"string"}

num_images = 500
original_dir = "/content/dataset/valid"
temp_dir = "/content/temp_subset"

# Get random image files
images = [f for f in os.listdir(f"{original_dir}/images")
          if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
selected = random.sample(images, min(num_images, len(images)))

# Create directories and copy files
os.makedirs(f"{temp_dir}/valid/images", exist_ok=True)
os.makedirs(f"{temp_dir}/valid/labels", exist_ok=True)

for img in selected:
    label = img.rsplit('.', 1)[0] + '.txt'
    shutil.copy(f"{original_dir}/images/{img}", f"{temp_dir}/valid/images/{img}")
    if os.path.exists(f"{original_dir}/labels/{label}"):
        shutil.copy(f"{original_dir}/labels/{label}", f"{temp_dir}/valid/labels/{label}")

# Create temp_data.yaml
nc_int = int(nc)
names_list = [name.strip().strip("'") for name in class_names.split(',')]

temp_data_string = f"""
train: /content/temp_subset/train/images
val: /content/temp_subset/valid/images
nc: {nc_int}
names: {names_list}
"""

with open("/content/temp_data.yaml", 'w') as f:
    f.write(temp_data_string.strip())

print("Created temp_data.yaml.")

# Clone WiseEye, python3.10-dev for tf, maken env_yolo11 and clone kris-himax ultralytics


In [ ]:
!git clone https://github.com/HimaxWiseEyePlus/YOLO11_on_WE2
%cd YOLO11_on_WE2

In [ ]:
!sudo apt install python3.10-dev

In [ ]:
!python3.10 -m venv --without-pip env_yolo11
!wget https://bootstrap.pypa.io/get-pip.py
!source env_yolo11/bin/activate && \
  which python && \
  python --version && \
  which pip && \
  python get-pip.py && \
  pip install tensorflow-cpu

In [ ]:
!source env_yolo11/bin/activate && \
    git clone https://github.com/kris-himax/ultralytics && \
    cd ultralytics && \
    pip install . && \
    cd ..

# Copy custom model to env_yolo11 and convert_tflite.py

In [ ]:
# Set your model name here
model_name = "best.pt" #@param {type:"string"}
imgsz = 256 #@param {type:"integer"}
data="/content/temp_data.yaml"

# Copy the .pt file to the YOLO11_on_WE2 directory
!cp /content/drive/MyDrive/yolo/{model_name} /content/YOLO11_on_WE2/

# Create the updated convert_tflite.py content
convert_script = f"""import ultralytics
ultralytics.checks()
from ultralytics import YOLO
model = YOLO('/content/YOLO11_on_WE2/{model_name}')
model.export(format='tflite', imgsz={imgsz}, int8=True, no_post=True, data='/content/temp_data.yaml')
"""

# Write the updated script to the file
with open('/content/YOLO11_on_WE2/examples/convert_tflite.py', 'w') as f:
    f.write(convert_script)

print(f"Updated convert_tflite.py with model name: {model_name}")
print("File contents:")
print(convert_script)

# Convert full_integer_quant_vela.tflite

In [ ]:
!source env_yolo11/bin/activate && \
  unset MPLBACKEND && \
  export MPLBACKEND=Agg && \
  python3.10 examples/convert_tflite.py

In [ ]:
!source env_yolo11/bin/activate && \
  pip install ethos-u-vela && \
  wget https://raw.githubusercontent.com/HimaxWiseEyePlus/ML_FVP_EVALUATION/main/vela/himax_vela.ini

In [ ]:
name = "best" #@param {type:"string"}

!source env_yolo11/bin/activate && \
  vela --accelerator-config ethos-u55-64 \
  --config himax_vela.ini \
  --system-config My_Sys_Cfg \
  --memory-mode My_Mem_Mode_Parent \
  --output-dir /content/YOLO11_on_WE2/{name}_saved_model/ \
  /content/YOLO11_on_WE2/best_saved_model/{name}_full_integer_quant.tflite

In [ ]:
!source env_yolo11/bin/activate && \
  vela --accelerator-config ethos-u55-64 \
  --config himax_vela.ini \
  --system-config My_Sys_Cfg \
  --memory-mode My_Mem_Mode_Parent \
  --output-dir /content/YOLO11_on_WE2/best_256_saved_model/ \
  /content/YOLO11_on_WE2/best_saved_model/best_full_integer_quant.tflite

# Zip and download the results

In [ ]:
from google.colab import files

name = "best" #@param {type:"string"}

# Zip the saved model directory
!zip -r /content/{name}_saved_model.zip /content/YOLO11_on_WE2/{name}_saved_model

# Download the zip file
files.download(f'/content/{name}_saved_model.zip')